### DATA FROM CATASTRO INSPIRE
___________________________________________________________________________________________________

Obteined through qGIS after mingling from a while with:
- WMS service: http://ovc.catastro.meh.es/cartografia/INSPIRE/spadgcwms.aspx
- WFS buildings : http://ovc.catastro.meh.es/INSPIRE/wfsBU.aspx?
- todos los serivicios INSPIRE: http://www.catastro.minhap.es/webinspire/index.html

*Notas: el uso de los canales WMS/WFS devuelven las capas antes de unificar con el programa Europeo Inspire, por lo que quizá sea la razón por la que no funcionan bien.

Existen varios GitHubs enfocados a consultas del catastro con python:
- **[pyCatastro](#https://github.com/gisce/pycatastro)**: permite realizar consultas en formato API. 
>>- No permite la descarga total de datos en función de municipio y provincia
>>- Devuelve diccionarios
>>- Para descargarme Madrid, debería obtener (1) Todas las vías, (2) Todas las siglas e iterar, haciendo mogollón de llamadas (not a good idea)

- **[catastro-lib-python](#https://github.com/sperea/catastro-lib-python)**: parece un antecersor del anterior. No se ha probado pues se dejó de actualizar en 2018
- **[Catastro Inspire Downloader](#https://github.com/geomatico/cidownloader)**: realizados por los mismos cartógrafos y desarrolladores que el complemento de qGIS, permite descargar datos en formato .geopackage
>>- Permite descargar según provincia, municipio y proyección
>>- Es poco consistente: al descargar Madrid a veces devuelve sólo datos de CadastralParcel o de BuildingParts. Comparando con los gmls incluidos en ZIPs que se pueden descargar a parte, los datos son incompletos. Posiblemente se un problema con la librería GDAL.


In [1]:
import pandas as pd
import numpy as np

import geopandas as gpd
import geojson
import folium.folium

import seaborn as sns
import matplotlib.pyplot as plt
import folium

%matplotlib inline

pd.set_option('display.max_rows', 1000)

### CAPAS DISPONIBLES

Dada el volumen de datos, la inspección de éstos se realiza sobre una parte de éstos

In [2]:
CATASTRO_PATH = '../data/raw/catastro'

# Dentro de los datos displibles hay 4 capas en formato geojson

building_df = gpd.read_file(f"{CATASTRO_PATH}/A.ES.SDGC.BU.28900.building.geojson", rows = 25000)
buildingParts_df = gpd.read_file(f"{CATASTRO_PATH}/A.ES.SDGC.BU.28900.buildingpart.geojson", rows = 25000) # pt 1
otherConstruction_df = gpd.read_file(f"{CATASTRO_PATH}/A.ES.SDGC.BU.28900.otherconstruction.geojson", rows = 25000)

cadastralParcel = gpd.read_file(f"{CATASTRO_PATH}/A.ES.SDGC.CP.28900.cadastralparcel.geojson", rows = 25000)
cadastralZoning = gpd.read_file(f"{CATASTRO_PATH}/A.ES.SDGC.CP.28900.cadastralzoning.geojson", rows = 25000)

Checkeo de uso de memoria

In [3]:
print(f"Building Layer total memory usage: \t\t\t{building_df.memory_usage(index=True).sum()/1000} \tKbytes")
print(f"Building Parts Layer total memory usage: \t\t{buildingParts_df.memory_usage(index=True).sum()/1000} \tKbytes")
print(f"Other Construction Layer total memory usage: \t\t{otherConstruction_df.memory_usage(index=True).sum()/1000} \t\tKbytes")

print(f"Cadastral Parcel Layer total memory usage: \t\t{cadastralParcel.memory_usage(index=True).sum()/1000} \tKbytes")
print(f"Cadastral Zoning Layer total memory usage: \t\t{cadastralZoning.memory_usage(index=True).sum()/1000} \tKbytes")

Building Layer total memory usage: 			4825.128 	Kbytes
Building Parts Layer total memory usage: 		2625.128 	Kbytes
Other Construction Layer total memory usage: 		774.16 		Kbytes
Cadastral Parcel Layer total memory usage: 		2200.128 	Kbytes
Cadastral Zoning Layer total memory usage: 		1269.136 	Kbytes


## 1.1 Building Layer

En qGIS esta capa representa la parte edificada de los solares.

In [4]:
building_df.shape

(25000, 25)

In [5]:
building_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 25 columns):
 #   Column                                   Non-Null Count  Dtype   
---  ------                                   --------------  -----   
 0   gml_id                                   25000 non-null  object  
 1   beginLifespanVersion                     25000 non-null  object  
 2   conditionOfConstruction                  25000 non-null  object  
 3   beginning                                25000 non-null  object  
 4   end                                      25000 non-null  object  
 5   endLifespanVersion                       13 non-null     object  
 6   informationSystem                        25000 non-null  object  
 7   reference                                25000 non-null  object  
 8   localId                                  25000 non-null  object  
 9   namespace                                25000 non-null  object  
 10  horizontalGeometryEstimate

In [6]:
building_df.isna().sum()

gml_id                                         0
beginLifespanVersion                           0
conditionOfConstruction                        0
beginning                                      0
end                                            0
endLifespanVersion                         24987
informationSystem                              0
reference                                      0
localId                                        0
namespace                                      0
horizontalGeometryEstimatedAccuracy            0
horizontalGeometryEstimatedAccuracy_uom        0
horizontalGeometryReference                    0
referenceGeometry                              0
currentUse                                   279
numberOfBuildingUnits                          0
numberOfDwellings                              0
numberOfFloorsAboveGround                  25000
documentLink                                   0
format                                         0
sourceStatus        

**GETTING RID OF USELESS COLUMNS**

In [7]:
# DROP NULL COLUMNS
# There are a couple of columns that do not offer any information

building_nullCols = ['numberOfFloorsAboveGround', 'endLifespanVersion']
building_linkCols = ['documentLink', 'format', 'informationSystem']
building_measureCols = ['horizontalGeometryEstimatedAccuracy', 'value_uom', 'sourceStatus',
                        'horizontalGeometryEstimatedAccuracy_uom', 'horizontalGeometryReference', 
                        'officialAreaReference']

building_df.drop(building_nullCols,    axis=1, inplace = True)
building_df.drop(building_linkCols,    axis=1, inplace = True)
building_df.drop(building_measureCols, axis=1, inplace = True)

# horizontalGeometryEstimatedAccuracy is always 0.1m accuracy

In [8]:
# CORRECTING GROSS FLOOR AREA
# officialAreaReference - value - value_uom are columns that refer to the same information

building_df.rename(columns={"value": "grossFloorArea"}, inplace = True)
# building_df.drop(['officialAreaReference'], axis=1, inplace = True)

In [9]:
# SIMPLIFYING DATE COLUMNS
# changes between them migt be because of difference between registration or CFO.
# end column has no sense in the context of this project

building_dateCols = ['beginLifespanVersion', 'beginning', 'end']

# ARE beginning and end the same

building_df['Equal_beg_end'] = building_df.apply(lambda x: x['beginning'] == x['end'], axis = 1)
building_df['Equal_beg_end'].value_counts()

True     24166
False      834
Name: Equal_beg_end, dtype: int64

Let's see the rest of DATES columns.
In the [bibliography](#http://www.catastro.minhap.es/webinspire/documentos/Conjuntos%20de%20datos.pdf) it says:
>- **beginLifespanVersion**: Fecha desde cuándo se ha dado de alta en la base de datos
catastral. 
>- **dateOfConstruction**: estructura que define la fecha de construcción. Está compuesta por dos atributos: bu-c**ore2d:beginning y bu-core2d:end**; los valores son las fechas de construcción de cada unidad constructiva, si hay más de una en el campo **“beginning” se incluye la más antigua y en el campo “end” la más moderna** Siempre se referencian al 1 de enero . 

**______**
Para este proyecto, sólo nos interesa la columna beginning

In [10]:
# Dont know what this means.. could be buildings that are demolished or abandoned
# Lets look to the first row reference = 000207800VK56E
# Sede Electronica del Catastro says that this building has been built in 2004, the *end* year
# Most plausible cause is that the builing is abandoned, in construction etc, is a phase of reconstruction

# conditionOfConstruction should be different
# Lets see that

c_functional, c_declined, c_ruin = 0, 0, 0

for ref in building_df[building_df['Equal_beg_end'] == False]['reference'].tolist() :
    if building_df[building_df['reference'] == ref]['conditionOfConstruction'].tolist()[0] == 'functional':
        c_functional += 1
    elif building_df[building_df['reference'] == ref]['conditionOfConstruction'].tolist()[0] == 'declined':
        c_declined += 1
    elif building_df[building_df['reference'] == ref]['conditionOfConstruction'].tolist()[0] == 'ruin':
        c_ruin += 1
    else:
        pass

print(f"FOR BUILDINGS WITH THE DIFFERENT BEGINNING AND END DATES. CONDITION OF CONSTRUCTION")
print(f"Functional : \t{c_functional}")
print(f"Declined : \t{c_declined}")
print(f"Ruins : \t{c_ruin}")

# There should be another category
# Buildings that may have another new cadastral reference for unknown reasons

FOR BUILDINGS WITH THE DIFFERENT BEGINNING AND END DATES. CONDITION OF CONSTRUCTION
Functional : 	833
Declined : 	0
Ruins : 	1


In [11]:
# LETS SEE CONDITION OF CONSTRUCTION COLUMN
building_df['conditionOfConstruction'].value_counts()

functional    24592
-               248
declined        109
ruin             51
Name: conditionOfConstruction, dtype: int64

In [12]:
# LETS ADD another category for demolished buildings, that reflects beginning != end
# por tanto, los edificios en donde se ha construido suelen son edificios funcionales
# 
# Como se ha dicho más arriba, sólo nos interesa la columna BEGINNING para efectos de este proyecto

building_df.drop(['Equal_beg_end', 'end', 'beginLifespanVersion'], axis = 1, inplace = True)

In [15]:
import datetime as dt

building_df['dateOfConstruction'] = building_df['beginning'].apply(lambda x: 
                                    dt.datetime.strptime(x,'%Y-%m-%dT%H:%M:%S'))

# Out of bounds nanosecond timestamp: 1640-01-01 00:00:00
# Pandas required YEARS to be inside de bound of 1670 - 2560,
# Because of the nature of data, datetime methods cannot be used for this case

def get_yearofConstruction(strng):
    """
    Input:  string
    Output: year as string
    """
    first_w = strng.split('T')[0]
    # Not using datetime from pandas, not valid for this case
    return first_w.split('-')[0]

building_df['yearOfConstruction'] = building_df['beginning'].apply(get_yearofConstruction)

In [16]:
building_df.drop(['beginning', 'dateOfConstruction'], axis = 1, inplace = True)

**____________________________________________________________________________________________________________**

In [17]:
# THERE ARE COLUMNS THAT MAY DUPLICATE INFORMATION
# To join with the rest of data, parcels, buildingparts... share an ID
# Which col is the ID ??

cols_id = ['gml_id', 'reference', 'localId', 'namespace']

# Reference == localID ?? 
# See if there are unique values, or repeteated values (this info has to be contrasted with the rest of geojson)

building_df[cols_id].describe()

# All elements are unique (as expected)
# SHOULD COINCIDE WITH PARCELS
# gml_id for later

,gml_id,reference,localId,namespace
count,25000,25000,25000,25000
unique,25000,25000,25000,1
top,ES.SDGC.BU.0152810VK4705A,0405401VK4700E,0405401VK4700E,ES.SDGC.BU
freq,1,1,1,25000


In [18]:
# Are reference and localID the same? 
building_df.apply(lambda x: x['reference'] == x['localId'], axis = 1).value_counts() # TRUE
# What is reference Geometry ?
building_df['referenceGeometry'].value_counts() # All true --> Dropping

# Dropping reference and namespace
building_df.drop(['reference', 'namespace', 'referenceGeometry'], axis = 1, inplace = True)

In [20]:
building_df.head(10)

,gml_id,conditionOfConstruction,localId,currentUse,numberOfBuildingUnits,numberOfDwellings,grossFloorArea,geometry,yearOfConstruction
0,ES.SDGC.BU. VK4700H,-,VK4700H,None,0,0,0,"POLYGON ((440433.629 4470953.961, 440432.170 4...",2000
1,ES.SDGC.BU.00006Z8VK4800A,-,00006Z8VK4800A,None,0,0,0,"POLYGON ((440020.830 4479859.707, 440020.064 4...",2000
2,ES.SDGC.BU.000200100VK48E,functional,000200100VK48E,2_agriculture,1,0,26,"POLYGON ((441657.574 4487050.292, 441658.164 4...",1968
3,ES.SDGC.BU.000200500VK56E,declined,000200500VK56E,3_industrial,1,0,646,"MULTIPOLYGON (((451584.920 4467181.410, 451586...",1995
4,ES.SDGC.BU.000205600VK56E,declined,000205600VK56E,3_industrial,1,0,202,"POLYGON ((451579.400 4467169.610, 451582.370 4...",1995
5,ES.SDGC.BU.000205700VK56E,declined,000205700VK56E,3_industrial,1,0,105,"POLYGON ((451576.440 4467163.260, 451576.930 4...",1995
6,ES.SDGC.BU.000205800VK56E,declined,000205800VK56E,3_industrial,1,0,120,"POLYGON ((451573.050 4467156.010, 451576.440 4...",1995
7,ES.SDGC.BU.000205900VK56E,declined,000205900VK56E,3_industrial,1,0,120,"POLYGON ((451569.670 4467148.760, 451570.230 4...",1995
8,ES.SDGC.BU.000206000VK56E,declined,000206000VK56E,3_industrial,1,0,120,"POLYGON ((451566.290 4467141.520, 451569.670 4...",1995
9,ES.SDGC.BU.000206100VK56E,declined,000206100VK56E,3_industrial,1,0,177,"POLYGON ((451560.380 4467128.720, 451561.530 4...",1995


**____________________________________________________________________________________________________________**

**____________________________________________________________________________________________________________**

## 1.2 Building Parts Layer

In [21]:
buildingParts_df.shape

(25000, 14)

In [22]:
buildingParts_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 14 columns):
 #   Column                                   Non-Null Count  Dtype   
---  ------                                   --------------  -----   
 0   gml_id                                   25000 non-null  object  
 1   beginLifespanVersion                     25000 non-null  object  
 2   conditionOfConstruction                  0 non-null      object  
 3   localId                                  25000 non-null  object  
 4   namespace                                25000 non-null  object  
 5   horizontalGeometryEstimatedAccuracy      25000 non-null  float64 
 6   horizontalGeometryEstimatedAccuracy_uom  25000 non-null  object  
 7   horizontalGeometryReference              25000 non-null  object  
 8   referenceGeometry                        25000 non-null  bool    
 9   numberOfFloorsAboveGround                25000 non-null  int64   
 10  heightBelowGround         

In [23]:
buildingParts_df.isna().sum()

gml_id                                         0
beginLifespanVersion                           0
conditionOfConstruction                    25000
localId                                        0
namespace                                      0
horizontalGeometryEstimatedAccuracy            0
horizontalGeometryEstimatedAccuracy_uom        0
horizontalGeometryReference                    0
referenceGeometry                              0
numberOfFloorsAboveGround                      0
heightBelowGround                              0
heightBelowGround_uom                          0
numberOfFloorsBelowGround                      0
geometry                                       0
dtype: int64

In [24]:
buildingParts_df.describe()

,horizontalGeometryEstimatedAccuracy,numberOfFloorsAboveGround,heightBelowGround,numberOfFloorsBelowGround
count,2.500000e+04,25000.000000,25000.000000,25000.00000
mean,1.000000e-01,3.778000,2.270880,0.75696
std,1.387807e-17,2.747504,2.871751,0.95725
min,1.000000e-01,0.000000,0.000000,0.00000
25%,1.000000e-01,1.000000,0.000000,0.00000
50%,1.000000e-01,3.000000,0.000000,0.00000
75%,1.000000e-01,6.000000,3.000000,1.00000
max,1.000000e-01,23.000000,30.000000,10.00000


In [25]:
buildingParts_df['namespace'].value_counts()

ES.SDGC.BU    25000
Name: namespace, dtype: int64

In [26]:
# DROPING USELESS 
buildingParts_measureCols = ['heightBelowGround_uom','horizontalGeometryEstimatedAccuracy',
                             'horizontalGeometryEstimatedAccuracy_uom', 'horizontalGeometryReference',
                            'referenceGeometry']
buildingParts_nullCols = ['conditionOfConstruction']
builingParts_repCols = ['namespace']

buildingParts_df.drop(buildingParts_measureCols, axis=1, inplace = True)
buildingParts_df.drop(buildingParts_nullCols,    axis=1, inplace = True)
buildingParts_df.drop(builingParts_repCols,      axis=1, inplace = True)

In [27]:
# GETTING YEAR OF CONSTRUCTION
buildingParts_df['yearOfConstruction'] = buildingParts_df['beginLifespanVersion'].apply(get_yearofConstruction)
buildingParts_df.drop(['beginLifespanVersion'], axis = 1, inplace = True)

In [28]:
# SEPARATING localId from its parts
def get_part(x):
    part_str = x.split('_')[1]
    return part_str.split('t')[1]

def get_ID(x):
    return x.split('_')[0]

buildingParts_df['locadID_parts'] = buildingParts_df['localId'].apply(get_part).astype(dtype = 'int64')
buildingParts_df['locadID'] = buildingParts_df['localId'].apply(get_ID)
buildingParts_df.drop(['localId'], axis = 1, inplace = True)

In [32]:
buildingParts_df.head(7)

# falta limpiar gml_id --> separar, como part ya se repite en localID, no hace falta

,gml_id,numberOfFloorsAboveGround,heightBelowGround,numberOfFloorsBelowGround,geometry,yearOfConstruction,locadID_parts,locadID
0,ES.SDGC.BU.000200100VK48E_part1,1,0,0,"POLYGON ((441657.574 4487050.292, 441658.164 4...",2004,1,000200100VK48E
1,ES.SDGC.BU.000200500VK56E_part1,1,0,0,"POLYGON ((451584.920 4467181.410, 451586.250 4...",2013,1,000200500VK56E
2,ES.SDGC.BU.000200500VK56E_part2,1,0,0,"POLYGON ((451612.580 4467215.370, 451618.650 4...",2013,2,000200500VK56E
3,ES.SDGC.BU.000205600VK56E_part1,1,0,0,"POLYGON ((451579.400 4467169.610, 451582.370 4...",2013,1,000205600VK56E
4,ES.SDGC.BU.000205700VK56E_part1,1,0,0,"POLYGON ((451576.440 4467163.260, 451576.930 4...",2013,1,000205700VK56E
5,ES.SDGC.BU.000205800VK56E_part1,1,0,0,"POLYGON ((451573.050 4467156.010, 451576.440 4...",2013,1,000205800VK56E
6,ES.SDGC.BU.000205900VK56E_part1,1,0,0,"POLYGON ((451569.670 4467148.760, 451570.230 4...",2013,1,000205900VK56E


**________________________**

**________________________**

## 1.3 Other Construction Layer

In [33]:
otherConstruction_df.shape

(13822, 7)

In [34]:
otherConstruction_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 13822 entries, 0 to 13821
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   gml_id                   13822 non-null  object  
 1   beginLifespanVersion     13822 non-null  object  
 2   conditionOfConstruction  0 non-null      object  
 3   localId                  13822 non-null  object  
 4   namespace                13822 non-null  object  
 5   constructionNature       13822 non-null  object  
 6   geometry                 13822 non-null  geometry
dtypes: geometry(1), object(6)
memory usage: 756.0+ KB


In [35]:
otherConstruction_df.isna().sum()

gml_id                         0
beginLifespanVersion           0
conditionOfConstruction    13822
localId                        0
namespace                      0
constructionNature             0
geometry                       0
dtype: int64

In [36]:
otherConstruction_df['conditionOfConstruction'].value_counts()

Series([], Name: conditionOfConstruction, dtype: int64)

In [37]:
otherConstruction_df[['beginLifespanVersion', 'constructionNature', 'localId']].describe()

,beginLifespanVersion,constructionNature,localId
count,13822,13822,13822
unique,2191,1,13822
top,2003-01-02T00:00:00,openAirPool,2740401VK4284B_PI.3063
freq,899,13822,1


In [38]:
# All data corresponds to open AIR POOLS
# I don't need this dataset to begin with
# BUT lets cleaned it

In [39]:
otherConstruction_Cols = ['conditionOfConstruction', 'constructionNature', 'namespace']
otherConstruction_df.drop(otherConstruction_Cols, axis = 1, inplace = True)

In [43]:
otherConstruction_df.head(3)

,gml_id,geometry,locadID,yearOfConstruction
0,ES.SDGC.BU.0006601VK3800E_PI.1,"POLYGON ((429964.360 4480456.150, 429961.960 4...",0006601VK3800E,2017
1,ES.SDGC.BU.0007201VK3800E_PI.2,"POLYGON ((429883.850 4480596.541, 429888.120 4...",0007201VK3800E,2003
2,ES.SDGC.BU.0007202VK3800E_PI.3,"POLYGON ((429967.989 4480563.520, 429968.469 4...",0007202VK3800E,2006


In [41]:
# Using same functions as in Building Parts
def get_PI(x):
    return x.split('_')[1]

# Separating localID

otherConstruction_df['locadID_PI'] = otherConstruction_df['localId'].apply(get_PI) #not really useful
otherConstruction_df['locadID'] = otherConstruction_df['localId'].apply(get_ID)

otherConstruction_df.drop(['localId', 'locadID_PI'], axis = 1, inplace = True)

In [42]:
# CLEANING DATE
otherConstruction_df['yearOfConstruction'] = otherConstruction_df['beginLifespanVersion'].apply(get_yearofConstruction)
otherConstruction_df.drop(['beginLifespanVersion'], axis = 1, inplace = True)

In [44]:
# CLEANING gml_id
otherConstruction_df['gml_id'] = otherConstruction_df['gml_id'].apply(get_ID)

In [45]:
otherConstruction_df.head(3)

,gml_id,geometry,locadID,yearOfConstruction
0,ES.SDGC.BU.0006601VK3800E,"POLYGON ((429964.360 4480456.150, 429961.960 4...",0006601VK3800E,2017
1,ES.SDGC.BU.0007201VK3800E,"POLYGON ((429883.850 4480596.541, 429888.120 4...",0007201VK3800E,2003
2,ES.SDGC.BU.0007202VK3800E,"POLYGON ((429967.989 4480563.520, 429968.469 4...",0007202VK3800E,2006


**________________________**

**________________________**

## 1.4 Other Construction Layer

In [46]:
cadastralParcel.shape

(25000, 11)

In [47]:
cadastralParcel.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   gml_id                      25000 non-null  object  
 1   areaValue                   25000 non-null  int64   
 2   areaValue_uom               25000 non-null  object  
 3   beginLifespanVersion        25000 non-null  object  
 4   endLifespanVersion          0 non-null      object  
 5   localId                     25000 non-null  object  
 6   namespace                   25000 non-null  object  
 7   label                       25000 non-null  object  
 8   nationalCadastralReference  25000 non-null  object  
 9   pos                         25000 non-null  object  
 10  geometry                    25000 non-null  geometry
dtypes: geometry(1), int64(1), object(9)
memory usage: 2.1+ MB


In [48]:
cadastralParcel.isna().sum()

gml_id                            0
areaValue                         0
areaValue_uom                     0
beginLifespanVersion              0
endLifespanVersion            25000
localId                           0
namespace                         0
label                             0
nationalCadastralReference        0
pos                               0
geometry                          0
dtype: int64

In [51]:
cadastralParcel['endLifespanVersion'].value_counts()

Series([], Name: endLifespanVersion, dtype: int64)

In [52]:
cadastralParcel.drop(['endLifespanVersion'], axis = 1, inplace = True)

In [53]:
cadastralParcel_measureCols = ['areaValue_uom', 'namespace']
cadastralParcel.drop(cadastralParcel_measureCols, axis = 1, inplace = True)

In [55]:
# CLEANING YEAR (same as 1.3 - 1.2 - 1.1)
cadastralParcel['yearOfConstruction'] = cadastralParcel['beginLifespanVersion'].apply(get_yearofConstruction)
cadastralParcel.drop(['beginLifespanVersion'], axis = 1, inplace = True)

In [56]:
cadastralParcel.head(4)

,gml_id,areaValue,localId,label,nationalCadastralReference,pos,geometry,yearOfConstruction
0,ES.SDGC.CP.000200500VK56E,1268,000200500VK56E,005,000200500VK56E,451607.03 4467199.27,"MULTIPOLYGON (((451599.360 4467174.940, 451584...",2013
1,ES.SDGC.CP.000205600VK56E,297,000205600VK56E,056,000205600VK56E,451592.44 4467170.83,"MULTIPOLYGON (((451593.000 4467163.250, 451579...",2013
2,ES.SDGC.CP.000205700VK56E,155,000205700VK56E,057,000205700VK56E,451587.93 4467161.77,"MULTIPOLYGON (((451590.040 4467156.910, 451576...",2013
3,ES.SDGC.CP.000205800VK56E,174,000205800VK56E,058,000205800VK56E,451584.62 4467155.04,"MULTIPOLYGON (((451586.650 4467149.650, 451573...",2013


In [ ]:
# is localID the same as nationalCadastralReference

